# Ollama & OpenWebUI in Google Colab\n\nDieses Notizbuch richtet ein Ollama-Sprachmodell ein und macht es über eine öffentliche OpenWebUI-Schnittstelle zugänglich.\n\n**Anleitung:**\n1. Stellen Sie sicher, dass Ihre Colab-Laufzeit auf eine GPU-Instanz eingestellt ist (`Laufzeit` > `Laufzeittyp ändern` > `T4 GPU`).\n2. Fügen Sie in Zelle 2 Ihren `ngrok`-Token und das gewünschte Modell ein.\n3. Führen Sie die Zellen nacheinander aus.

In [ ]:
#@markdown Führen Sie diese Zelle aus, um alle erforderlichen Abhängigkeiten zu installieren.\n#@markdown > ⏳ **Dies kann einige Minuten dauern.**\n%%capture\nprint("⏳ Installation von Ollama, OpenWebUI und ngrok wird gestartet...")\n!pip install open-webui pyngrok nest-asyncio\n!curl -fsSL https://ollama.com/install.sh | sh\nprint("✅ Installation abgeschlossen!")

## 2. Konfiguration und Start\n\nGeben Sie hier Ihre Konfiguration ein.\n\n💡 **Hinweis zu den Modellen:** Große Modelle wie `gpt-oss:20b` (ca. 12 GB) benötigen viel RAM und Speicherplatz. Auf Standard-Colab-Instanzen kann dies zu Problemen führen. Wenn Sie auf Schwierigkeiten stoßen, versuchen Sie es mit einem kleineren Modell wie `llama3:8b` (ca. 5 GB) oder `phi3` (ca. 2.5 GB).\n\n⚠️ **Sicherheitswarnung:** Das direkte Einfügen von geheimen Schlüsseln (wie Ihrem `ngrok`-Token) wird nicht empfohlen. Die sicherste Methode ist die Verwendung von **Colab Secrets** (Schlüssel-Symbol in der Seitenleiste).

In [ ]:
#@markdown ---#@markdown ### Konfiguration\n#@markdown Geben Sie das Ollama-Modell an, das Sie verwenden möchten.\nMODEL_NAME = "gpt-oss:20b" #@param {type:"string"}\n#@markdown Geben Sie Ihren ngrok Authtoken ein (finden Sie unter [dashboard.ngrok.com](https://dashboard.ngrok.com/get-started/your-authtoken)).\nNGROK_AUTHTOKEN = ""  #@param {type:"string"}\n#@markdown ---\nimport os\nimport asyncio\nimport nest_asyncio\nfrom pyngrok import ngrok\nimport urllib.request\n\nnest_asyncio.apply()\n\nasync def wait_for_service(url, timeout=180):\n    print(f"⏳ Überprüfe die Erreichbarkeit von {url} (Timeout: {timeout}s)...")\n    start_time = asyncio.get_event_loop().time()\n    while True:\n        try:\n            with urllib.request.urlopen(url, timeout=5) as response:\n                if response.status >= 200 and response.status < 300:\n                    print(f"✅ Dienst unter {url} ist erreichbar.")\n                    return True\n        except Exception:\n            pass\n        if (asyncio.get_event_loop().time() - start_time) > timeout:\n            print(f"❌ Zeitüberschreitung beim Warten auf {url}.")\n            return False\n        await asyncio.sleep(5)\n\nasync def run_ollama():\n    if NGROK_AUTHTOKEN:\n        ngrok.set_auth_token(NGROK_AUTHTOKEN)\n        print("✅ ngrok Authtoken konfiguriert.")\n    else:\n        print("⚠️ Kein ngrok Authtoken angegeben. Tunnel könnte fehlschlagen.")\n\n    with open('ollama.log', 'w') as f: f.write('')\n    with open('webui.log', 'w') as f: f.write('')\n\n    print("⏳ Starte Ollama-Server...")\n    ollama_process = await asyncio.create_subprocess_shell('ollama serve > ollama.log 2>&1')\n    await asyncio.sleep(5)\n\n    print(f"⏳ Lade das Modell '{MODEL_NAME}' herunter. Dies kann einige Zeit dauern...")\n    pull_process = await asyncio.create_subprocess_shell(f'ollama pull {MODEL_NAME}')\n    await pull_process.wait()\n    if pull_process.returncode != 0:\n        print(f"❌ Fehler beim Herunterladen des Modells '{MODEL_NAME}'. Überprüfen Sie den Modellnamen und die ollama.log.")\n        return\n    print(f"✅ Modell '{MODEL_NAME}' erfolgreich heruntergeladen!")\n\n    print("⏳ Starte OpenWebUI-Server...")\n    os.environ['OLLAMA_BASE_URL'] = 'http://127.0.0.1:11434'\n    webui_process = await asyncio.create_subprocess_shell('open-webui --host 0.0.0.0 --port 8080 > webui.log 2>&1')\n\n    server_ready = await wait_for_service("http://127.0.0.1:8080")\n\n    if server_ready:\n        try:\n            public_url = ngrok.connect(8080)\n            print(f"\n✅ OpenWebUI ist bereit! Öffnen Sie diesen Link in Ihrem Browser:\n{public_url}")\n        except Exception as e:\n            print(f'\n❌ Fehler beim Starten von ngrok: {e}')\n            print("Überprüfen Sie den Authtoken und die Log-Dateien.")\n    else:\n        print("\n❌ OpenWebUI konnte nicht gestartet werden. Führen Sie die nächste Zelle aus, um die Logs zu überprüfen.")\n    \n    print("\nℹ️ Die Server laufen im Hintergrund. Diese Zelle muss aktiv bleiben, um die Verbindung aufrechtzuerhalten.")\n\nasyncio.run(run_ollama())

---## 3. Fehlerbehebung\n\nWenn Sie Probleme haben (z.B. eine weiße Seite sehen), führen Sie die folgende Zelle aus, um die Log-Dateien der Server anzuzeigen. Dies kann Aufschluss über mögliche Fehler geben.

In [ ]:
print("--- OLLAMA LOG (ollama.log) ---")
with open('ollama.log', 'r') as f:
    print(f.read())

print("\n--- OPENWEBUI LOG (webui.log) ---")
with open('webui.log', 'r') as f:
    print(f.read())